In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
import json



In [ ]:
clo_info=pd.read_excel(r'C:\Users\Radhe Shyam\Desktop\clo engine\clo_info.xlsx',sheet_name="Tranche_info")
interest_waterfall_info=pd.read_excel(r'C:\Users\Radhe Shyam\Desktop\clo engine\clo_info.xlsx',sheet_name="Interest_waterfall")


,Class,Tranche type,Preliminary rating,Balance,Overcollateralization (%),Par subordination (%),Benchmark rate,Spread or coupon,coverage_test_group,Rank
0,A-1,current_pay,AAA (sf),330.00,166.67,40.0,SOFR,1.35,A/B,1.0
1,A-2,current_pay,AAA (sf),27.50,153.85,35.0,SOFR,1.65,A/B,1.0
2,B,current_pay,AA (sf),60.50,131.58,24.0,SOFR,1.75,A/B,1.0
3,C (deferrable),deferrable,A (sf),33.00,121.95,18.0,SOFR,2.05,C,2.0
4,D-1a (deferrable),deferrable,BBB+ (sf),16.50,117.65,15.0,SOFR,2.85,D,3.0
5,D-1b (deferrable),deferrable,BBB (sf),11.00,114.94,13.0,SOFR,3.35,D,3.0
6,D-2 (deferrable),deferrable,BBB- (sf),8.25,112.99,11.5,SOFR,4.15,D,3.0
7,E (deferrable),deferrable,BB- (sf),19.25,108.70,8.0,SOFR,5.85,E,4.0
8,Subordinated notes,residual,NR,48.98,NaN,NaN,NaN,Residual,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
coverage_test_info=pd.read_excel(r'C:\Users\Radhe Shyam\Desktop\clo engine\clo_info.xlsx',sheet_name="Coverage_test")
coverage_test_info


,Class,Actual O/C (%),Min. O/C required (%),Min. I/C required (%)
0,A/B,131.58,121.58,120.0
1,C,121.95,113.95,115.0
2,D,112.99,106.99,110.0
3,E,108.70,103.70,NaN
4,Interest diversion test(i),108.70,104.20,NaN


In [3]:
class CLODataManager:
    def __init__(self, file_path='clo_data.json'):
        self.file_path = file_path
        self.load_data()
    
    def load_data(self):
        """Load or initialize shared CLO data"""
        try:
            with open(self.file_path, 'r') as f:
                self.data = json.load(f)
        except FileNotFoundError:
            self.data = self.initialize_defaults()
    
    def initialize_defaults(self):
        return {
            "deal_info": {

                "initial_portfolio_value": 0,
                "current_portfolio_value": 0,
                "reinvestment_period_end": 20,
                "portfolio_was":3.36,
                "first_coupon_date":0
            },
            "deferred_interest": {},
            # "payment_history": [],
            "coverage_test_history": [],
            # "cash_balances": {
            #     "available_interest": 0,
            #     "available_principal": 0,
            "reserve_accounts": {}
            }
        # }
    
    def update_tranche_balance(self,tranche_name, new_balance):
        self.data["tranches"][tranche_name]["current_balance"] = new_balance
    
    def add_deferred_interest(self,period,tranche_name, amount):
        self.data["deferred_interest"][tranche_name].append({
            "period": period,
            "timestamp": datetime.now().isoformat(),
            # "type": payment_type,
            # "beneficiary": beneficiary,
            "amount": amount
        })

    def update_reserve_account(self,period,amount):
        self.data["reserve_accounts"][period]=amount

        # if tranche_name not in self.data["deferred_interest"]:
        #     self.data["deferred_interest"][tranche_name] = 0
        # self.data["deferred_interest"][tranche_name] += amount
    
    # def record_payment(self, period, payment_type, beneficiary, amount):
    #     self.data["payment_history"].append({
    #         "period": period,
    #         "timestamp": datetime.now().isoformat(),
    #         "type": payment_type,
    #         "beneficiary": beneficiary,
    #         "amount": amount
    #     })
    
    def save_data(self):
        with open(self.file_path, 'w') as f:
            json.dump(self.data, f, indent=2)

In [ ]:
class Interestwaterfallengine:

    def __init__(self,tranche_info,interest_payment_waterfall,coverage_tests,sofr_curve):
        self.tranche_info=tranche_info
        self.interest_payment_waterfall=interest_payment_waterfall
        self.sofr_curve=sofr_curve
        self.coverage_tests=coverage_tests
        self.deal_info=CLODataManager()
        
        
    def interest_waterfall(self):
        priority=self.interest_payment_waterfall.set_index("Payment")["Condition"].to_dict()
        spread_info=self.tranche_info.set_index("Class")["Spread or coupon"].to_dict()
        tranche_balance_info=self.tranche_info.set_index("Class")["Balance"].to_dict()
        tests_info=self.coverage_tests.set_index("Class")["O/C required","I/C required"].to_dict(orient="index")
        interest_tracking_dict={}
        current_principal=self.deal_info["deal_info"]["current_portfolio_value"]
        interest_received=current_principal*self.deal_info["deal_info"]["portfolio_was"]
        for i in priority.keys():
            if priority[i] in ["fee/must_pay"]:
                payment_due=current_principal*spread_info[i]
                interest_tracking_dict[i]=payment_due if interest_received >= payment_due else "default"
                interest_received-=payment_due
            elif priority[i] in ["interest"]:

                # calculate tranche interest u dumb fuck not the whole principal
                payment_due=tranche_balance_info[i]*spread_info[i]
                interest_tracking_dict[i]=payment_due if interest_received >= payment_due else "default"
                interest_received-=payment_due

            elif priority[i] in ["coverage_test"]:
                rank=self.tranche_info.loc[self.tranche_info["coverage_test_group"] == priority[i],
                                        "rank"].unique()[0]
                principal_balances_of_rank=self.tranche_info.loc[self.tranche_info["Rank"] <= rank,
                                        "Balance"].sum()
                interest_due_of_rank=principal_balances_of_rank*self.tranche_info.loc[self.tranche_info["Rank"] <= rank,
                                        "Spread or coupon"].sum()             
                tranche_oc_required=tests_info[i]["O/C required"]
                tranche_ic_required=tests_info[i]["I/C required"]
                current_tranche_oc=current_principal/principal_balances_of_rank
                current_tranche_ic=interest_received/interest_due_of_rank
                if current_tranche_oc>tranche_oc_required:
                    cure_required=principal_balances_of_rank*tranche_oc_required
                    cure_required=cure_required-current_principal
                    tranche_order=self.tranche_info["Class"].dropna()
                    for tranche in tranche_order:
                        if tranche_balance_info[tranche]==0:
                            continue
                        else:
                            tranche_balance_info[tranche]-=min(interest_received,cure_required)
                            interest_received-=min(interest_received,cure_required)
                elif current_tranche_ic>tranche_ic_required:
                    cure_required=interest_due_of_rank*tranche_ic_required
                    cure_required=cure_required-interest_received
                    tranche_order=self.tranche_info["Class"].dropna()
                    for tranche in tranche_order:
                        if tranche_balance_info[tranche]==0:
                            continue
                        else:
                            tranche_balance_info[tranche]-=min(interest_received,cure_required)
                            interest_received-=min(interest_received,cure_required)

            elif priority[i] in ["deferrable_interest"]:
                pass
                    
                            
                    
                    


                
    # def coverage_test(self,tranche,):
    #     pass
    def accrued_interest():
        pass


    pass


In [2]:
import json
with open("clo.json", 'r') as f:
    data = json.load(f)

In [3]:
data

{'deal_info': {'starting_portfolio_amt': 1000000,
  'current_portfolio_amt': 0,
  'first_coupon_date': 0,
  'reinvestment_period_end': 10}}

In [26]:
tranche_info=clo_info[~clo_info["Tranche type"].isin(["current_pay", "residual"])]
tranche_info=tranche_info.set_index("Class")[["Balance","Spread or coupon"]].to_dict(orient="index")
tranche_info

{'C (deferrable)': {'Balance': 33.0, 'Spread or coupon': 2.05},
 'D-1a (deferrable)': {'Balance': 16.5, 'Spread or coupon': 2.85},
 'D-1b (deferrable)': {'Balance': 11.0, 'Spread or coupon': 3.35},
 'D-2 (deferrable)': {'Balance': 8.25, 'Spread or coupon': 4.15},
 'E (deferrable)': {'Balance': 19.25, 'Spread or coupon': 5.85}}

In [10]:
# # clo_info = clo_info.dropna(subset=["coverage_test_group"])   
# clo_info=clo_info["Class"].dropna()
# # clo_info["rank"] = clo_info["coverage_test_group"].rank(method="dense").astype(int)
# # clo_info[["coverage_test_group","rank"]]


# # Rank=clo_info.loc[clo_info["Rank"] <= 1,
# #                                         "Balance"].sum()
# # Rank
# # clo_info


ss=interest_waterfall_info.set_index("Payment")["Condition"].to_dict()

ss

{'Taxes and fees, and then administrative expenses (capped).': 'fee/must_pay',
 'To the payment of the base management fee.': 'fee/must_pay',
 'Class A-1 interest.': 'interest',
 'Class A-2 interest.': 'interest',
 'Class B interest.': 'interest',
 'Class A/B coverage tests(i)(ii).': 'coverage_test',
 'Class C interest.': 'deferrable_interest',
 'Class C coverage tests(i)(ii).': 'coverage_test',
 'Class C deferred interest.': 'accrued_interest',
 'Class D-1a interest.': 'deferrable_interest',
 'Class D-1b interest.': 'deferrable_interest',
 'Class D-2 interest.': 'deferrable_interest',
 'Class D coverage tests(i)(ii).': 'coverage_test',
 'Class D-1a deferred interest.': 'accrued_interest',
 'Class D-1b deferred interest.': 'accrued_interest',
 'Class D-2 deferred interest.': 'accrued_interest',
 'Class E interest.': 'deferrable_interest',
 'Class E coverage test(i)(ii).': 'coverage_test',
 'Class E deferred interest.': 'accrued_interest',
 'Effective date ratings confirmation. If it is

In [ ]:
class interest_tracking():
    def __init__(self,filename,clo_info):
        self.file=filename
        self.clo_info=clo_info
    
    def load(self):
        try:
            with open(self.file,"w") as f:
                self.data=json.load(self.file)
        except:
            self.data=self.initialize_deferrable_tranches()

    def initialize_deferrable_tranches(self):
        tranche_info=self.clo_info[~self.clo_info["Tranche type"].isin(["current_pay", "residual"])]
        



    def update_accrued_info(self):
        
        
